In [4]:
#parsing and pattern match

In [31]:
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

In [32]:
human = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 看看 | 找找 | 想找点
活动 = 乐子 | 玩的
"""

In [33]:
import random

In [34]:
def create_grammar(host, split='=', line_split='\n'):#对grammar_str转变成对应二维列表
    grammar= {}
    for line in host.split(line_split):
        if not line.strip():continue#strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split("|")]
    return grammar

In [35]:
def generate (gram,target):#递归生成句子
    if target in gram:
        expand = random.choice(gram[target])
    else:return target 
    return "".join(generate(gram , t) for t in expand)

In [36]:
grammar = create_grammar(host)
#print(grammar)
generate(gram=grammar, target='host')#调用函数

'小朋友,您好我是8号,请问你要耍一耍打猎吗？'

In [11]:
 #Language Model

In [9]:
import pandas as pd
content = pd.read_csv('./movie_comments.csv')#读取文件
content.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [10]:
articles = content['comment'].tolist()#将数组或者矩阵转换成列表

In [11]:
import re#正则表达式
def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)#去除符号

In [12]:
articles_clean = [''.join(token(str(a)))for a in articles]#逐个列表元素进行数据清洗

In [15]:
with open('article_c.txt', 'w',encoding='utf-8') as f:#文件中写入articles_clean,,encoding='utf-8'指定编码
    for a in articles_clean:
        f.write(a + '\n')

In [16]:
import jieba
from collections import Counter#Counter计数器，统计字符出现的个数

In [17]:
def cut(string): return list(jieba.cut(string))#对string分词返回列表

In [24]:
TOKEN = []
for i, line in enumerate(open('article_c.txt',encoding='utf-8')):#一行一行读
    #if i % 100 == 0: print(i)
    TOKEN += cut(line)#TOKEN：articles_clean中有所有词
    # replace 10000 with a big number when you do your homework. 
    #if i > 10000: break    

words_count = Counter(TOKEN)
words_count.most_common(10)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MSI\AppData\Local\Temp\jieba.cache
Loading model cost 1.116 seconds.
Prefix dict has been built succesfully.


[('的', 328262),
 ('\n', 261497),
 ('了', 102420),
 ('是', 73106),
 ('我', 50338),
 ('都', 36255),
 ('很', 34712),
 ('看', 34022),
 ('电影', 33675),
 ('也', 32065)]

In [25]:
#TOKEN = [str(t) for t in TOKEN]#这句没用吧？
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]#？？？
TOKEN_2_GRAM[:20]

['吴京意淫',
 '意淫到',
 '到了',
 '了脑残',
 '脑残的',
 '的地步',
 '地步看',
 '看了',
 '了恶心',
 '恶心想',
 '想吐',
 '吐\n',
 '\n首映礼',
 '首映礼看',
 '看的',
 '的太',
 '太恐怖',
 '恐怖了',
 '了这个',
 '这个电影']

In [26]:
words_count_2 = Counter(TOKEN_2_GRAM)#2个字的连词
words_count_2.most_common(10)

[('了\n', 22033),
 ('的\n', 10158),
 ('的电影', 8604),
 ('啊\n', 8389),
 ('看的', 7106),
 ('\n我', 7029),
 ('都是', 6335),
 ('让人', 5284),
 ('的故事', 4673),
 ('看了', 4585)]

In [27]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / len(TOKEN_2_GRAM)#word1, word2连词概率
    else:
        return 1 / len(TOKEN_2_GRAM)

In [28]:
def get_probablity(sentence):
    words = cut(sentence)
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        
        probability = prob_2(word, next_)#2-gram language models；p(s)=p(w1)p(w2| w1)p(w3| w1w2)?p(wl| w1?wl?1)
        
        sentence_pro *= probability
    
    return sentence_pro

In [29]:
get_probablity('小明今天抽奖抽到一台苹果手机')#这句话的概率

4.343268699453012e-40

In [37]:
for sen in [generate(gram=grammar, target='host') for i in range(10)]:
    print('human: {} with Prb: {}'.format(sen, get_probablity(sen)))

human: 你好我是5号,请问你要耍一耍赌博吗？ with Prb: 5.911758394169286e-79
human: 你好我是5327号,您需要玩一玩打牌吗？ with Prb: 3.009686337828214e-70
human: 你好我是888号,您需要玩一玩打猎吗？ with Prb: 3.009686337828214e-70
human: 先生,你好我是9号,您需要耍一耍喝酒吗？ with Prb: 7.012689214050025e-89
human: 小朋友,您好我是2号,您需要玩一玩喝酒吗？ with Prb: 4.6652133792371254e-82
human: 小朋友,您好我是4号,您需要耍一耍打猎吗？ with Prb: 2.384314332777008e-89
human: 女士,您好我是272号,请问你要耍一耍打猎吗？ with Prb: 3.463196329124419e-95
human: 小朋友,您好我是3号,您需要玩一玩打猎吗？ with Prb: 5.287241829802075e-82
human: 女士,你好我是6号,请问你要玩一玩赌博吗？ with Prb: 2.493501356969583e-92
human: 你好我是5号,您需要玩一玩打猎吗？ with Prb: 3.792204785663551e-68
